# Studie av tekster 

Her for barnebøker

fra mail

For litt siden skrev du (13. mars):

«Du hadde noen spørsmål knyttet til ordlistene og hva som kan gjøres videre. Når det gjelder geografi, land og byer, har vi en oversikt. Likeså for navn, men ikke hvor navnene kommer fra. Det kan vi se på etter hvert - hvilke fornavn og etternavn fins, og hva forteller dem om kulturelt mangfold? Vi har metoder under utvikling for å få et bedre grep om dem»

Jeg kunne ha god bruk for at disse ble lagt inn, slik at jeg får en oversikt over hvilke geografiske steder utenom Norge som blir nevnt i materialet – og hvor ofte – samt hvilke fornavn og etternavn som blir brukt.

 

13. mars skrev du: «Hvis excel er noe du kan tenke deg å bruke, kan jeg generere hele excelbøker med oversikt over enkeltordene og verkene de finnes i.»

Det høres ut som et veldig nyttig arbeidsverktøy for meg.

 

13. mars skrev du også:

«Mulig det greieste er å ta ut skolebøker som et eget korpus ved hjelp av Sofies lister.»

Jeg skal ikke bruke skolebøkene til noe, det greieste for meg er å vite at de overhode ikke er med i korpuset, så hvis det lar seg gjøre å ta dem ut, ville det være fint.

 

In [1]:
import json
import sqlite3
import requests
import pandas as pd
import dask
import pandas as pd
import dhlab.module_update as mu
mu.update('mods_meta')
import mods_meta as mm
import dhlab.nbtext as nb



Updated file `D:\Documents\GitHub\Miscellaneous-Notebooks\Barnebokinstituttet_prosjekt\mods_meta.py`

In [11]:
def remove_school(meta):
    newmeta = meta[~meta['653- - -a'].str.contains('lærebøker', na=False)]
    newmeta = newmeta[~newmeta['653- - -a'].str.contains('grunnskolen', na=False)]
    return newmeta

In [55]:
def aggregate_corpus(urns, step = 300):
    babi = pd.DataFrame()
    for i in range(0, len(urns), step):
        print(i)
        df = nb.get_aggregated_corpus(urns[i:i+step])
        babi = nb.frame(pd.concat([babi, df], axis=1, sort=False).sum(axis=1))
    return babi

# Henter ut data

In [2]:
barnebøker_fra_2015 = mm.meta_natbib(children=True, yearfrom =2015, limit=1000)

In [56]:
barnebok_urns = [z[1] for z in barnebøker_fra_2015]

Henter ut all tekst

In [5]:
corpus_text = nb.get_corpus_text(barnebok_urns)

In [211]:
corpus_text.dropna(axis=0, how='all')

,2019021477022,2019072307202,2018072507217,2018062507081,2018072407012,2018081407171,2018082207048,2018061807573,2019011448520,2018061107082,...,2018071607137,2018020548069,2018081407102,2018061107008,2018071707621,2018071607094,2018072507066,2018072507254,2018060807148,2018103007239
.,1023.0,2810.0,5227.0,8705.0,1850.0,610.0,117.0,183.0,225.0,71.0,...,107.0,776.0,106.0,263.0,3649.0,569.0,65.0,3574.0,5184.0,948.0
",",634.0,2131.0,2954.0,5420.0,705.0,740.0,70.0,351.0,116.0,53.0,...,41.0,635.0,31.0,127.0,1974.0,148.0,38.0,2421.0,3701.0,962.0
»,525.0,681.0,1724.0,2544.0,3.0,23.0,5.0,9.0,43.0,29.0,...,1.0,79.0,1.0,81.0,9.0,40.0,1.0,739.0,26.0,691.0
«,525.0,686.0,1729.0,2547.0,2.0,21.0,5.0,12.0,45.0,31.0,...,2.0,80.0,NaN,83.0,9.0,39.0,NaN,739.0,26.0,703.0
og,392.0,1114.0,2509.0,2493.0,7.0,136.0,58.0,59.0,39.0,3.0,...,18.0,489.0,NaN,92.0,882.0,220.0,35.0,1973.0,2141.0,501.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Booooiimrmnrmgggg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Boiinngg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Boiiiinnnggg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
BHÆftSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


# Her ser vi på ordlister


Henter ut ordlister fra Kristin.

fra mail:  
Handikappet/funksjonshemmet
Blind, svaksynt, tykk, tynn, døv, tunghørt, hørselshemmet, klumpfot, albino
 
Seksuell legning
trans, transe, homofil, homse, lesbisk, lesbe, bi, bifil
 
Egennavn
Fornavn, etternavn
 
Nasjonalitet
Pakistansk, arabisk, afrikansk, grønlender, grønlandsk, islandsk, polsk, russer, russisk, amerikaner, hindu, japaner, asiat
Finnes det en base som angir nasjonalitet?
 
Geografisk område
Norden, Europa, Asia, Amerika, Afrika, Australia, Antarktis, Arktis, Ural
Finnes det en base over land? Men dette går kanskje under kategorien Egennavn?
 
Språk – pt kommer jeg på disse, men dersom du har en base med flere språk sier jeg ikke nei til det:
engelsk, pakistansk, urdu, svensk, dansk, finsk, islandsk, grønlandsk, samisk, fransk, tysk, polsk, litauisk, russisk, italiensk, spansk, castillano, portugisisk, kinesisk, japansk, tamil, arabisk, swahili
 
Religion
kristendom, hinduisme, islam, muhammedanisme, taoisme, jødedom, koptisk, katolisisme
 
Andre kultur-markører
Urfolk, indianer, araber, afrikaner, amerikaner, europeer, nordisk, samisk
 

In [7]:
ordlister = """ 
diff: blind, svaksynt, tykk, tynn, døv, tunghørt, hørselshemmet, klumpfot, albino;
seksualitet: trans, transe, homofil, homse, lesbisk, lesbe, bi, bifil;
nasjonalitet: pakistansk, arabisk, afrikansk, grønlender, grønlandsk, islandsk, polsk, russer, russisk, amerikaner, hindu, japaner, asiat;
geografi: Norden, Europa, Asia, Amerika, Afrika, Australia, Antarktis, Arktis, Ural;
språk: engelsk, pakistansk, urdu, svensk, dansk, finsk, islandsk, grønlandsk, samisk, fransk, tysk, polsk, litauisk, russisk, italiensk, spansk, castillano, portugisisk, kinesisk, japansk, tamil, arabisk, swahili;
religion: kristendom, hinduisme, islam, muhammedanisme, taoisme, jødedom, koptisk, katolisisme;
kulturmarkør: urfolk, indianer, araber, afrikaner, amerikaner, europeer, nordisk, samisk;
"""

In [8]:
ordgrupper = {x.split(':')[0]:x.split(':')[1].split(', ') for x in [y.strip() for x in ordlister.split('\n') for y in x.split(';') if y.strip() != '']}

In [9]:
ordgrupper.keys()

dict_keys(['diff', 'seksualitet', 'nasjonalitet', 'geografi', 'språk', 'religion', 'kulturmarkør'])

In [59]:
barnemeta = mm.metadata(barnebok_urns)

In [62]:
barnebøker_urn = remove_school(nb.frame(barnemeta))

In [63]:
barnebøker_urn

,100-1- -a,245-1-0-a,260- - -c,653- - -a,lang,year,245-1-0-b,245-1-3-a,245-1-3-b
2015090906138,"Daynes, Katie.",NaN,2015,"barnebøker, drager, draker, lettlest",nno,2015,NaN,NaN,NaN
2015091006002,"Punter, Russell.",NaN,2015,"barnebøker, lettlest",nno,2015,NaN,NaN,NaN
2015091006003,"Punter, Russell.",NaN,2015,"barnebøker, katter, lettlest",nno,2015,NaN,NaN,NaN
2015091006004,"Likar, Gudrun.",Prinsesse Fibi og draken,2015,"barnebøker, drager, draker, lettlest, prinsesser",nno,2015,NaN,NaN,NaN
2015091006005,"Berge, Atle",Matias Maradona,2015,"barnebøker, fotball, lettlest",nno,2015,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2019011448517,NaN,NaN,[2015],NaN,nob,2015,NaN,NaN,NaN
2019011448533,NaN,NaN,cop. 2015,"barnebøker, billedbøker, norsk, ord, pekebøker...",nob,2015,NaN,NaN,NaN
2019011448562,NaN,NaN,2015,NaN,nob,2015,NaN,NaN,NaN
2019052107102,NaN,NaN,2015,NaN,nob,2015,NaN,NaN,NaN


In [82]:
barnebøker_urn.to_excel('barnebøker_korpus.xls')

In [65]:
babi_urnlist = list(barnebøker_urn.index)

In [67]:
babi_urnlist[:10]

['2015090906138',
 '2015091006002',
 '2015091006003',
 '2015091006004',
 '2015091006005',
 '2015091506021',
 '2015091506022',
 '2015091506023',
 '2015091506024',
 '2015091506131']

# DF og TF

In [20]:
corpus_text

,2019021477022,2019072307202,2018072507217,2018062507081,2018072407012,2018081407171,2018082207048,2018061807573,2019011448520,2018061107082,...,2018071607137,2018020548069,2018081407102,2018061107008,2018071707621,2018071607094,2018072507066,2018072507254,2018060807148,2018103007239
.,1023.0,2810.0,5227.0,8705.0,1850.0,610.0,117.0,183.0,225.0,71.0,...,107.0,776.0,106.0,263.0,3649.0,569.0,65.0,3574.0,5184.0,948.0
",",634.0,2131.0,2954.0,5420.0,705.0,740.0,70.0,351.0,116.0,53.0,...,41.0,635.0,31.0,127.0,1974.0,148.0,38.0,2421.0,3701.0,962.0
»,525.0,681.0,1724.0,2544.0,3.0,23.0,5.0,9.0,43.0,29.0,...,1.0,79.0,1.0,81.0,9.0,40.0,1.0,739.0,26.0,691.0
«,525.0,686.0,1729.0,2547.0,2.0,21.0,5.0,12.0,45.0,31.0,...,2.0,80.0,NaN,83.0,9.0,39.0,NaN,739.0,26.0,703.0
og,392.0,1114.0,2509.0,2493.0,7.0,136.0,58.0,59.0,39.0,3.0,...,18.0,489.0,NaN,92.0,882.0,220.0,35.0,1973.0,2141.0,501.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Booooiimrmnrmgggg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Boiinngg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Boiiiinnnggg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
BHÆftSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [16]:
m = nb.frame(mm.metadata(list(corpus_text.loc['homo'].dropna().index)))
m

,100-1- -a,245-1-0-a,260- - -c,653- - -a,lang,year,245-1-0-b
2014111907535,"Holm, Dagny",Zeppelin,2007,"grunnskolen, k06, lesebøker, lærebøker, norsk",nno,2015,NaN
2018051448017,"Green, John","Will Grayson, Will Grayson",2015,NaN,nob,2015,NaN
2018060807044,"Lyga, Barry",Game,2015,NaN,nob,2015,NaN
2018060807049,"Johnsen, Janne Aasebø",Kamuflasje,2015,"forelskelse, norsk, oppvekst, romaner, ungdoms...",nob,2015,NaN
2018061807034,"Green, John",Papirbyer,2015,NaN,nob,2015,NaN
2018061807069,"Larsson, Åsa",Nidstangen,2015,"barnelitteratur, spenning",nob,2015,NaN
2018062507069,"Taylor, Laini",Mørk engel,2015,"amerikansk, fantasy, ungdomslitteratur",nob,2015,NaN
2018062607030,"Aakeson, Kim Fupz",Jeg er William,2015,NaN,nob,2015,NaN
2018070507023,"Sævareid, Heidi",Slipp hold,2015,"kjærlighet, kontroll, makt, skjønnlitteratur, ...",nob,2015,NaN
2018070507099,"Oseman, Alice",Solospill,2015,"blogging, forelskelse, internett, tenåringer, ...",nob,2015,NaN


In [21]:
m_no_school = remove_school(m)

In [23]:
m_no_school.index

Index(['2018051448017', '2018060807044', '2018060807049', '2018061807034',
       '2018061807069', '2018062507069', '2018062607030', '2018070507023',
       '2018070507099', '2018070507114', '2018071607133', '2018072607141',
       '2018072607233', '2018072607271', '2018072607298', '2018072607320',
       '2018091207011', '2018091407049', '2018101107031', '2018102207052',
       '2018112177044', '2019030177042', '2018060748070'],
      dtype='object')

In [25]:
corpus_no_school = corpus_text[[int(x) for x in m_no_school.index]]

In [218]:
cns = corpus_no_school.dropna(axis=0, how='all')

In [219]:
cns

,2018051448017,2018060807044,2018060807049,2018061807034,2018061807069,2018062507069,2018062607030,2018070507023,2018070507099,2018070507114,...,2018072607271,2018072607298,2018072607320,2018091207011,2018091407049,2018101107031,2018102207052,2018112177044,2019030177042,2018060748070
.,4867.0,10154.0,3646.0,5997.0,2187.0,7677.0,1177.0,4285.0,6819.0,4356.0,...,490.0,4344.0,5799.0,6299.0,9075.0,12300.0,10137.0,2347.0,7610.0,5828.0
",",4729.0,5926.0,2243.0,5337.0,806.0,7066.0,691.0,3730.0,3683.0,2506.0,...,267.0,3061.0,4669.0,5520.0,6413.0,8704.0,7230.0,1560.0,4269.0,4733.0
»,1202.0,3158.0,16.0,2183.0,32.0,2168.0,490.0,1076.0,2182.0,1542.0,...,47.0,518.0,2330.0,1755.0,2380.0,4743.0,4477.0,1275.0,1472.0,2613.0
«,1205.0,3162.0,16.0,2187.0,32.0,2168.0,491.0,1078.0,2192.0,1542.0,...,41.0,516.0,2327.0,1767.0,2384.0,4764.0,4476.0,1271.0,1473.0,2616.0
og,2486.0,2865.0,1090.0,2702.0,641.0,3580.0,424.0,2132.0,2302.0,1378.0,...,210.0,2530.0,2236.0,2314.0,3426.0,4630.0,3967.0,798.0,2892.0,2223.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dansert,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Danseri,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CRAYStfM,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AUlP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
corpus_no_school.to_csv('ord_i_korpus.csv')

In [187]:
check_word = lambda word, corpus = corpus_no_school: corpus.loc[[w for w in ordgrupper[word] if w in corpus.index]].count(axis=1)

In [117]:
ordgrupper = {key: [x.strip() for x in ordgrupper[key]] for key in ordgrupper}
ordgrupper

{'diff': ['blind',
  'svaksynt',
  'tykk',
  'tynn',
  'døv',
  'tunghørt',
  'hørselshemmet',
  'klumpfot',
  'albino'],
 'seksualitet': ['trans',
  'transe',
  'homofil',
  'homse',
  'lesbisk',
  'lesbe',
  'bi',
  'bifil'],
 'nasjonalitet': ['pakistansk',
  'arabisk',
  'afrikansk',
  'grønlender',
  'grønlandsk',
  'islandsk',
  'polsk',
  'russer',
  'russisk',
  'amerikaner',
  'hindu',
  'japaner',
  'asiat'],
 'geografi': ['Norden',
  'Europa',
  'Asia',
  'Amerika',
  'Afrika',
  'Australia',
  'Antarktis',
  'Arktis',
  'Ural'],
 'språk': ['engelsk',
  'pakistansk',
  'urdu',
  'svensk',
  'dansk',
  'finsk',
  'islandsk',
  'grønlandsk',
  'samisk',
  'fransk',
  'tysk',
  'polsk',
  'litauisk',
  'russisk',
  'italiensk',
  'spansk',
  'castillano',
  'portugisisk',
  'kinesisk',
  'japansk',
  'tamil',
  'arabisk',
  'swahili'],
 'religion': ['kristendom',
  'hinduisme',
  'islam',
  'muhammedanisme',
  'taoisme',
  'jødedom',
  'koptisk',
  'katolisisme'],
 'kulturmarkør

In [188]:
check_word('seksualitet')

trans      0
transe     5
homofil    8
homse      6
lesbisk    4
lesbe      2
bi         1
bifil      1
dtype: int64

In [189]:
check_word('seksualitet', corpus_text)

trans       2
transe     32
homofil    16
homse       8
lesbisk     8
lesbe       3
bi         69
bifil       3
dtype: int64

In [150]:
barnebøker_urn

,100-1- -a,245-1-0-a,260- - -c,653- - -a,lang,year,245-1-0-b,245-1-3-a,245-1-3-b
2015090906138,"Daynes, Katie.",NaN,2015,"barnebøker, drager, draker, lettlest",nno,2015,NaN,NaN,NaN
2015091006002,"Punter, Russell.",NaN,2015,"barnebøker, lettlest",nno,2015,NaN,NaN,NaN
2015091006003,"Punter, Russell.",NaN,2015,"barnebøker, katter, lettlest",nno,2015,NaN,NaN,NaN
2015091006004,"Likar, Gudrun.",Prinsesse Fibi og draken,2015,"barnebøker, drager, draker, lettlest, prinsesser",nno,2015,NaN,NaN,NaN
2015091006005,"Berge, Atle",Matias Maradona,2015,"barnebøker, fotball, lettlest",nno,2015,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2019011448517,NaN,NaN,[2015],NaN,nob,2015,NaN,NaN,NaN
2019011448533,NaN,NaN,cop. 2015,"barnebøker, billedbøker, norsk, ord, pekebøker...",nob,2015,NaN,NaN,NaN
2019011448562,NaN,NaN,2015,NaN,nob,2015,NaN,NaN,NaN
2019052107102,NaN,NaN,2015,NaN,nob,2015,NaN,NaN,NaN


In [197]:
writer = pd.ExcelWriter('boklister_tema.xlsx', engine='xlsxwriter')
for i in ordgrupper:
    barnebøker_urn.loc[[str(x) for x in corpus_no_school.loc[check_word(i).index].dropna(axis=1, how='all').columns]].to_excel(writer, sheet_name=i)
writer.save()

In [196]:
corpus_no_school.loc[check_word('seksualitet').index].dropna(axis=1, how='all').fillna(0)

,2018051448017,2018060807044,2018061807034,2018070507023,2018070507099,2018070507114,2018071607133,2018072607141,2018072607233,2018072607298,2018091207011,2018091407049,2018101107031,2018102207052,2019030177042
trans,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
transe,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
homofil,13.0,0.0,0.0,0.0,2.0,25.0,1.0,0.0,13.0,0.0,0.0,2.0,2.0,0.0,4.0
homse,3.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0
lesbisk,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lesbe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
bi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
bifil,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
konks = lambda w,urner = [x[1] for x in barnebøker_fra_2015]: [ ' '.join([str(y) for y in x]) for x in mm.konk(word=w,urns=urner)]

In [41]:
all_ordene = [y.strip() for x in ordgrupper for y in ordgrupper[x]]
#all_ordene

In [42]:
konkordanser = {w:konks(w) for w in all_ordene}

In [45]:
nb.frame(konkordanser)

,0,1,2,3,4,5,6,7,8,9,...,757,758,759,760,761,762,763,764,765,766
blind,2018062507081 ville gjort meg enda mer blind ....,2018062507081 Tror du jeg er blitt blind ? »,2019011448520 som drar med seg en blind mann 1...,"2018031248066 . Seeren er seiv en blind , og f...",2018031248066 Han var en blind . Nåbyrn . En l...,"2018031248066 han så på henne , blind eller ik...",2018031248066 sent . Han var en blind . Hun ha...,"2018031248066 Vettskremt . Til og med blind , ...",2018061207220 jeg skjønner at du er blind som ...,2018061207220 ganske godt til å være blind . »...,...,None,None,None,None,None,None,None,None,None,None
svaksynt,2018091205064 på at keiserkobraen var like sva...,2018072607298 Jessica var svaksynt og var av d...,2018072607298 . En trettenåring som på svaksyn...,2018102207047 . Dessverre er jeg ganske svaksy...,2018050748058 None svaksynt adj mal-voyant m ( -,2018060807164 eneste i verden som er svaksynt ...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
tykk,"2019021477022 det en flaske med en tykk , gula...",2019021477022 fantastiske blandingen . Mikstur...,2018072507217 et kraftig trebord lå en tykk None,"2018072507217 ned i skjul bak en tykk trerot ,...",2018072507217 - og stanset ved en tykk bok i s...,"2018062507081 stund tittet en lav , tykk ung f...","2018062507081 med stort hode og en tykk , muse...","2018062507081 , en solid fem centimeter tykk s...","2018062507081 meg , som vognhjul gjennom tykk ...",2018062507081 mot tinningene . Luffen var tykk...,...,None,None,None,None,None,None,None,None,None,None
tynn,2019021477022 . Hun var faktisk veldig tynn og...,"2019072307202 en tråd , mer ei tynn remse med ...","2019072307202 opp til munnen . Ei tynn , firfi...",2019072307202 fort og med en veldig tynn og hv...,"2019072307202 . Den var grå og tynn , omtrent ...",2019072307202 . Men jeg var akkurat tynn nok t...,"2018072507217 » hørte han plutselig en tynn , ...",2018072507217 et stykke unna steg en tynn stri...,2018072507217 han bort og hentet en tynn notat...,"2018072507217 over ham , og en tynn solstråle ...",...,2018072507254 til læreren ble til en tynn stre...,"2018072507254 , var en høy , tynn mann med sol...",2018072507254 sto en seng med en tynn madrass ...,2018072507254 smør under en tynn ostebit . Sag...,2018072507254 ? » spurte han med tynn stemme ....,2018060807148 ned og etterlot seg en tynn stri...,2018060807148 hendene i siden . En tynn film a...,2018060807148 Det siste kom som en tynn bønn .,"2018060807148 , og munnen ble en tynn strek in...",2018060807148 henne . Jentekroppen var så tynn...
døv,2018062507081 så jeg ble lamslått og døv .,"2018081607070 hand kan en gjete , døv kjempær ...",2018061207220 oss andre - er nesten døv nå ! »,"2018060807051 en blind nordmann , en døv dansk...",2018072607320 « Jeg er jo ikke døv . »,2018101007059 i Norge . Å være døv betyr at du...,"2018101007059 høre . Om du er døv , uttrykker ...",2018090607026 like gjerne kunne ha vært døv . ...,2018090607026 merket det om hun ble døv eller ...,2018060807170 Hadde eksplosjonen gjort ham døv...,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
indianer,"2018061107007 siden av gata ? En indianer , næ...","2018060748068 Kvinnen var indianer , mens mann...",2018060748068 var stengt når du var indianer o...,2018061807053 vært innfødt amerikaner eller mu...,2018050748058 2 indianer None,2018050748058 2 indianer y indifférence [ edif...,2018050748058 None indianer s Indien m ( -,2018030248032 enten det var å leke indianer el...,"2018070507073 ut som en hedning og indianer , ...",2019032277143 seiv både som farget og indianer...,...,None,None,None,None,None,None,None,None,None,None
araber,"2018061807053 sett en ekte , levende araber ? »",2018061807053 . Helt seriøst . En araber og en...,2018061807053 « Var han araber ? »,"2018061807053 Nei , han var ikke araber . Han ...",2018050748058 ] s m / f araber None

## Lag katalog over ordlistene

In [224]:
babi = nb.frame(corpus_no_school.fillna(0).sum(axis=1))

In [226]:
babi = babi[babi[0] != 0]

In [227]:
nb.frame(babi).to_csv('babi_ordliste.csv', header=None)

In [120]:
oversikt ={key: nb.frame_sort(
    nb.frame(
        {w:int(babi.loc[w].sum()) for w in ordgrupper[key] if w in babi.index },
        'frekv'
    )
) for key in ordgrupper.keys()}

In [134]:
oversikt_trunkert ={key: nb.frame_sort(
    nb.frame(
        {w:int(babi.loc[[ws for ws in babi.index if ws.startswith(w)]].sum()) for w in ordgrupper[key] },
        'frekv'
    )
) for key in ordgrupper.keys()}

# Oversikt over frekvens av ord

Her vil vi også kunne lage dokumentfrekvens ganske greit. "Om litt" :)

In [131]:
oversikt['diff']

,frekv
tynn,95
tykk,48
blind,41
døv,13
svaksynt,2
tunghørt,0
albino,0


In [121]:
oversikt['seksualitet']

,frekv
homofil,62
homse,11
transe,5
lesbisk,4
lesbe,2
bifil,2
bi,1
trans,0


In [122]:
oversikt['nasjonalitet']

,frekv
polsk,24
russisk,7
afrikansk,6
amerikaner,4
arabisk,2
asiat,1
pakistansk,0
grønlandsk,0
islandsk,0
russer,0


In [123]:
oversikt['religion']

,frekv
katolisisme,1
kristendom,0
hinduisme,0
islam,0
taoisme,0
jødedom,0
koptisk,0


In [124]:
oversikt['geografi']

,frekv
Amerika,21
Australia,16
Europa,12
Afrika,7
Asia,4
Antarktis,2
Arktis,1
Norden,0
Ural,0


In [125]:
oversikt['nasjonalitet']

,frekv
polsk,24
russisk,7
afrikansk,6
amerikaner,4
arabisk,2
asiat,1
pakistansk,0
grønlandsk,0
islandsk,0
russer,0


In [126]:
oversikt['språk']

,frekv
engelsk,68
polsk,24
spansk,22
tysk,19
fransk,17
kinesisk,10
italiensk,9
russisk,7
svensk,6
japansk,3


In [127]:
oversikt['kulturmarkør']

,frekv
amerikaner,4
nordisk,1
urfolk,0
indianer,0
araber,0
afrikaner,0
europeer,0
samisk,0


In [128]:

writer = pd.ExcelWriter('oversikt.xlsx', engine='xlsxwriter')

for i in oversikt:
    oversikt[i].to_excel(writer, sheet_name = i)

writer.save()

In [136]:

writer = pd.ExcelWriter('oversikt_trunkert.xlsx', engine='xlsxwriter')

for i in oversikt_trunkert:
    oversikt_trunkert[i].to_excel(writer, sheet_name = i)

writer.save()